**Heidelberg University**

**Data Science  Group**
    
Prof. Dr. Michael Gertz  

Ashish Chouhan, Satya Almasian, John Ziegler, Jayson Salazar, Nicolas Reuter
    
January 16, 2024
    
Natural Language Processing with Transformers

Winter Semster 2023/2024     
***

# **Assignment 4: Question Answering**
**Due**: Monday, January 29, 2024, 2pm, via [Moodle](https://moodle.uni-heidelberg.de/course/view.php?id=19251)



### **Submission Guidelines**

- Solutions need to be uploaded as a **single** Jupyter notebook. You will find several pre-filled code segments in the notebook, your task is to fill in the missing cells.
- For the written solution, use LaTeX in markdown inside the same notebook. Do **not** hand in a separate file for it.
- Download the .zip file containing the dataset but do **not** upload it with your solution.
- It is sufficient if one person per group uploads the solution to Moodle, but make sure that the full names of all team members are given in the notebook.

***
Member Name: Ye Tao, Yong Wu, Ziwei Liu, Guangdeng Liang

## **Task 1: Retrieval Augmented Generation (RAG)** ( 4.5 + 3 + 4 + 3 + 1.5 = 16 points)

In this task, we look at using the open source `Llama-13b-chat` model for creating a RAG system. You must first apply for access to Llama 2 models via [this](https://ai.meta.com/resources/models-and-libraries/llama-downloads/) form (access is typically granted within a few hours). etrieval augmented generation you also need to request to use the model on Hugging Face by going to the [model](https://huggingface.co/meta-llama/Llama-2-13b-chat-hf) card. ***Note that the emails you provide for your Hugging Face account must match the email you used to request Llama 2.***

The final piece that you need is a Hugging Face authentication token. You can find such a token by going to the `setting` in your Hugging Face profile, under the `Access Token` menu you can generate a new token.

To store the document you will need a free Pinecone [API key](https://app.pinecone.io/).
Make sure you have these pieces ready before starting to work on this task.

----
When ready, let's start by downloading the necessary packages.

It is advised to proceed with this notebook with a GPU (if you are on Colab make sure that a GPU environment is activated.)


Place all the access tokens in the `.env` file and upload it to the working directory (if you are running this notebook locally, you can change the path to fit your working directory). Please use the following format:


```
HF_AUTH= "Hugging Face Authentication Key"
PINECONE_API_KEY="Pincone API Key"
PINECONE_ENVIRONMENT="Pinecone Environment"
```

Run the cell below to load the access tokens into the environment variables.

In [1]:
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
from dotenv import load_dotenv

# load environment variables from .env file
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

In [ ]:
%pip install litellm
%pip install -qU trulens_eval pydantic fastapi kaleido python-multipart uvicorn cohere openai tiktoken "llama-index"
%pip install transformers
%pip install sentence-transformers
%pip install pinecone-client
%pip install datasets
%pip install accelerate
%pip install einops
%pip install langchain
%pip install xformers
%pip install bitsandbytes
%pip install matplotlib seaborn tqdm
%pip install chromadb
%pip install evaluate
%pip install rouge_score
%pip install bert_score



## Subtask 1.1: Data Preparation



We need a collection of documents to perform our retrieval on. To make it closer to your final project, you will be downloading and using a subset of the LangChain documentation. We get some of the `.html` files located on the site. The code below will download all HTML files from the links on the webpage into a `docs` directory. `-l1` limits the download to only the first level of depth.


In [2]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!wget -r -l1 -A.html -P docs https://api.python.langchain.com/en/stable/langchain_api_reference.html

 The docs are going to be used as input text for answering questions that a normal language model might not be aware of (LangChain docs is not necessarily part of its training data of Llama2). We can use LangChain itself to process these docs. Use the [ReadTheDocsLoader](https://python.langchain.com/docs/integrations/document_loaders/readthedocs_documentation) to load the docs from the `docs` folder.

 At the time of creating this notebook, there  `423` documents were downloaded. However, since the documentation is being updated regularly this number might be different for you.

In [3]:
from langchain.document_loaders import ReadTheDocsLoader
#### your code ####
loader = ReadTheDocsLoader("docs")
docs = loader.load()
#### your code ####
len(docs)

417

Let's take a look at one of the documents. You see that LangChain has created a `Document` object. Look at the example below and fill in the cells to print out the text content and URL of the page (the URL of the page should starts with `https://`).

In [ ]:
docs[10]

Document(page_content='langchain_nvidia_trt 0.0.1¶\nlangchain_nvidia_trt.llms¶\nClasses¶\nllms.StreamingResponseGenerator(client,\xa0...)\nA Generator that provides the inference results from an LLM.\nllms.TritonTensorRTError\nBase exception for TritonTensorRT.\nllms.TritonTensorRTLLM\nTRTLLM triton models.\nllms.TritonTensorRTRuntimeError\nRuntime error for TritonTensorRT.', metadata={'source': 'docs/api.python.langchain.com/en/stable/nvidia_trt_api_reference.html'})

In [ ]:
#### your code ####
page_content=docs[10].page_content
page_url=docs[10].metadata['source']
page_url = page_url.replace("docs/", "https://")
#### your code ####
print(page_content)
print(page_url)

langchain_nvidia_trt 0.0.1¶
langchain_nvidia_trt.llms¶
Classes¶
llms.StreamingResponseGenerator(client, ...)
A Generator that provides the inference results from an LLM.
llms.TritonTensorRTError
Base exception for TritonTensorRT.
llms.TritonTensorRTLLM
TRTLLM triton models.
llms.TritonTensorRTRuntimeError
Runtime error for TritonTensorRT.
https://api.python.langchain.com/en/stable/nvidia_trt_api_reference.html


As you can imagine the documents can be long and if multiple of them are required as context to answer questions, we need to take the document lengths into account.
This is due to the fact that language models do not have unlimited context span. In our case, we plan to use Llama2 for this project, where the maximum token limit is 4096. This limit is not only the input but also takes the generated output into account, moreover, you need to leave room for the query and instructions as well. Therefore, it is important to chunk the longer documents into smaller-sized fragments.

Based on your use case and how many contexts you plan to feed into the model the length of these fragments will differ.
In this case, we choose to assign 2000 tokens to context and choose to generate the answer from 5 context fragments, which leaves us with 400 tokens per context fragment as the maximum chunk size.

To count the number of tokens in a chunk, we need to load the correct tokenizer for Llama2. Fill the code cell below to load the correct tokenizer and use it to complete the function that counts the number of tokens per given chunk.

**Hint:** you need to use your Hugging Face authentication token to load the tokenizer.

In [2]:
#If you get an error here during the first import from the `transformers` package, restart the kernel and try again.
#### your code ####
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-13b-chat-hf", token=os.getenv('HF_AUTH'))

#### your code ####

In [3]:
def token_len(text):
  #### your code ####
    return len(tokenizer.encode(text))
    #### your code ####

Count the number of tokens for all documents and use it to compute minimum, maximum, and average token count statistics across all documents. Depending on how the documentation is updated by the time you run the cell below the numbers might slightly differ.

In [ ]:
#### your code ####
token_counts = [token_len(doc.page_content) for doc in docs]
min_tokens = min(token_counts)
avg_tokens = sum(token_counts) / len(token_counts)
max_tokens = max(token_counts)
#### your code ####
print(f"""Min: {min_tokens}
Avg: {avg_tokens}
Max: {max_tokens}""")

Min: 49
Avg: 4093.021582733813
Max: 38563


Now we will use LangChain's built-in chunking functionality to split the text into smaller chunks. LangChain offers a variety of text splitters that you can check out [here](https://api.python.langchain.com/en/latest/langchain_api_reference.html#module-langchain.text_splitter).
Use the general-purpose splitter that splits text by recursively looking at characters. Use this class to split the text into 400 token-sized chunks, where the length of each chunk is computed based on the `token_len` function. The length is not the only criterion for splitting, if any of these separators `'\n\n', '\n', ' ', ''` is encountered, we will have a new chunk.
Since splitting only based on maximum length might result in incoherent chunks for every consecutive chunk, let the chunk overlap by 50 tokens. This way,  we preserve some of the previous context while chunking.

In [4]:
#### your code ####
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size =400,
    length_function =token_len,
    chunk_overlap =50,
    separators=['\n\n', '\n', ' ', '']
)
#### your code ####

In [ ]:
chunks = text_splitter.split_text(docs[100].page_content)
len(chunks)

26

In [ ]:
token_len(chunks[0])

316

The next step is to apply the splitting function to all the documents in our corpus and to save our chunks in a logical way. We also want to assign a unique ID to each chunk so we know which part of the documentation they come from. In the end, the corpus should be transformed into a list of dictionaries of the following format:


```
[
    {
        "id": "glossary-0",
        "text": "first chunk of the document glossary",
        "source": "https://langchain.readthedocs.io/en/latest/glossary.html"
    },
    {
        "id": "glossary-1",
        "text": "second chunk of glossary",
        "source": "https://langchain.readthedocs.io/en/latest/glossary.html"
    }
    ...
]
```

Construct the IDs by taking the name of the page before the suffix `.html` and appending a chronological number indicating which chunk it is.


In [ ]:
from tqdm.auto import tqdm

documents = []

for doc in tqdm(docs):
  #### your code ####
    url = doc.metadata['source'].replace("docs/", "https://")
    uid = url.split('/')[-1].replace('.html', '')
    chunks = text_splitter.split_text(doc.page_content)
    for i, chunk in enumerate(chunks):
        documents.append({
            "id": f"{uid}-{i}",
            "text": chunk,
            "source": url
        })

  #### your code ####
len(documents) # once again this value might differ based on how the LangChain documentation is updated

  0%|          | 0/417 [00:00<?, ?it/s]

6062

For the next steps, we require a `DataFrame`.

In [ ]:
import pandas as pd
data = pd.DataFrame(documents)
data.head()

,id,text,source
0,langchain_api_reference-0,langchain 0.1.3¶\nlangchain.agents¶\nAgent is ...,https://api.python.langchain.com/en/latest/lan...
1,langchain_api_reference-1,Base Single Action Agent class.\nagents.agent....,https://api.python.langchain.com/en/latest/lan...
2,langchain_api_reference-2,[Deprecated] Chat Agent.[Deprecated] Chat Age...,https://api.python.langchain.com/en/latest/lan...
3,langchain_api_reference-3,[Deprecated] Agent for the MRKL chain.[Deprec...,https://api.python.langchain.com/en/latest/lan...
4,langchain_api_reference-4,Parses a message into agent action/finish.\nag...,https://api.python.langchain.com/en/latest/lan...


#### ${\color{red}{Comments\ 1.1}}$

${\color{red}{⚠️Comments\ begin⚠️}}$


```
✅ Point distribution ✅
- 0.5 point if the documents are correctly loaded with `ReadTheDocsLoader`.
    - 0.5 / 0.5 Points since the function is exactly the same
- 0.5 point if the page content and URL is extracted from the `Document` object.
    - 0.5 / 0.5 Points. The implementation is exact the one in the sample solution.
- 0.5 point if the tokenizer is correctly initialized.
    - 0.5 / 0.5 Points. The loading function accesses the environment variable directly rather than saving it to a variable beforehand, but the functionality is the same
- 0.25 point if the `token_len` function is correct.
    - 0.25 / 0.25 Points. The function is the same a inm the sample solution
- 0.75 point to compute the minimum, maximum and average length of all documents.
    - 0.75 / 0.75 Points. The actual values differ (probably due to a later point of execution) but the computation is correct. 
- 1 point for using the correct text splitter and using the correct parameters. For each mistake, deduct 0.25 point.
    - 1/ 1 Points. The Splitter is initialized correctly and the application of it is also correct.
- 1 point for converting the documents into chunks and list of dictionaries.
    - 0.75 / 1 Points. The function is practically correct, but the source field in the dataframe is not as in the sample solution,as it still contains the https part and not the docs/ part

Overall:

4.25 / 4.5 Points

```


${\color{red}{⚠️Comments\ end⚠️}}$

## Subtask 1.2: Document Embedding Pipeline


In this task, we initialize the embedding pipeline to transform the chunks into vector embeddings using Hugging Face and LangChain. These embeddings are used for similarity search between the query and the chunks to retrieve the most relevant chunks.
  We will use the `sentence-transformers/all-MiniLM-L6-v2` model for embedding, which is a rather small model that you can easily run on Colab. Initialize the model using `HuggingFaceEmbeddings` to use Hugging Face via Langchain. The encoding batch size should be 32, and make sure that the model is placed on the correct device, otherwise, this can take a long time.

In [5]:
from torch import cuda
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
import os
import pinecone
from tqdm import tqdm

In [6]:
embedding_model = 'sentence-transformers/all-MiniLM-L6-v2'
device = 'cuda:0'
docs = [
    "An example document",
    "A second document as an example"
]
### your code ###
embed_model = HuggingFaceEmbeddings(
    model_name=embedding_model,
    model_kwargs={'device': device},
    encode_kwargs={'batch_size': 32}
)
### your code ###

/root/miniconda3/envs/nlp/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


Embed the example documents using the model you created and check the output.
The output should be a list of lists, containing the embeddings.

In [ ]:
### your code ###
embeddings = embed_model.embed_documents(docs)
### your code ###
print("number of docs:",len(embeddings))
print("dimension of docs:",len(embeddings[0]))

number of docs: 2
dimension of docs: 384


Now we use the embedding pipeline created above to store the embeddings in a Pinecone vector index. First, lets setup the Pinecone environment, collect your API key and environment name from the environment variables, and initiate Pinecone with them.

In [ ]:
### your code ###
pinecone = pinecone.Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))
### your code ###

Initialize the index `rag-assignment` inside Pinecone. Use the cosine similarity as similarity metric. Keep in mind that if you run this multiple times on a free tier, where only one index is allowed, you need to remove the index created to make room for a new one (Pinecone index gets archived automatically after 14 days of inactivity).

In [ ]:
index_name = 'rag-assignment'
### your code ###
from pinecone import PodSpec

if index_name not in pinecone.list_indexes().names():
    pinecone.create_index(
        name=index_name,
        dimension=len(embeddings[0]),
        metric='cosine',
        spec=PodSpec(
            environment="gcp-starter"
        )
    )
### your code ###

Lets take a look at the index you created. As of now the index should be empty but have the correct embedding dimension.

In [ ]:
index_name = 'rag-assignment'
index = pinecone.Index(index_name)
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

Process the dataset in batches of `32` and push the vectors to the Pinecone index. Your index should include the IDs and embeddings for each chunk. As metadata, pass the original text as `text` and the URL as `source` (no need to add the `https`). We use this metadata later to retrieve the original text.

In [ ]:
batch_size = 32

for i in tqdm(range(0, len(data), batch_size)):
    ### your code ###
    batch_docs = documents[i:i+batch_size]
    ids = [doc['id'] for doc in batch_docs]
    texts = [doc['text'] for doc in batch_docs]
    embeds = embed_model.embed_documents(texts)
    vectors = []
    for id, embed, doc in zip(ids, embeds, batch_docs):
        vector_data = {
            "id": id,
            "values": embed,  # Convert numpy array to list
            "metadata": {
                "text": doc['text'], 
                "source": doc['source'].replace('https://', '')
            }
        }
        vectors.append(vector_data)
    index.upsert(vectors=vectors)
    ### your code ###

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 190/190 [01:30<00:00,  2.10it/s]


Now if we look at the index statistics we should have vectors of dimension `384`.

In [ ]:
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0601,
 'namespaces': {'': {'vector_count': 6010}},
 'total_vector_count': 6010}

'total_vector_count': 6010 less than len(documents): 6062 because of duplicates  
Number of duplicate rows: 104
Checked the data and found it was https://api.python.langchain.com/en/latest/langchain_api_reference.html duplicated.  
This is split into 52 chunks, which means there are 52 duplicate items not in the index.  
So 6010 is correct.

In [ ]:
duplicates = data[data.duplicated(subset='id', keep=False)]
print(f"Number of duplicate rows: {len(duplicates)}")

Number of duplicate rows: 104


#### ${\color{red}{Comments\ 1.2}}$

${\color{red}{⚠️Comments\ begin⚠️}}$


```
✅ Point distribution ✅
- 0.5 point if `HuggingFaceEmbeddings` is correclty initialized.
    - 0.5 / 0.5 Points. It is correctly initialized, but the encode kwargs are missing the device option. This does not yield different results but might take longer in execution.
- 0.5 point if examples are correctly processed with `HuggingFaceEmbeddings`.
    - 0.5 / 0.5 Points. The examples yield the correct results.
- 0.5 point if Pinecone is initialized with the enviroment variables.
    - 0.25 / 0.5 Points. Pincone is initialized using another method and not using the environment variable for the environment name. 
- 0.5 if the Pinecone index is correctly created with the metric specified.
    - 0.5 / 0.5 Points. Pinecone index is initiallized using the cosine similarity. 
- 1 point if the passages are converted to vectors and are pushed into Pinecone.
    1 / 1 Points. It seems like the function does practically the same even though the list comprehension is defined a little bit different


Overall:

1.75 / 2 Points
```


${\color{red}{⚠️Comments\ end⚠️}}$

## Subtask 1.3: Text Generation Pipeline


So far we have our index ready and a way to find the most similar chunks to our query. Now, we need a way to generate the answer from the retrieved chunks. For this purpose, we use the `text-generation` pipeline from Hugging Face (refer to the Hugging Face [tutorial](https://moodle.uni-heidelberg.de/pluginfile.php/1286642/mod_resource/content/1/HuggingFace.ipynb)) and load it into LangChain using a wrapper.

In [7]:
from torch import cuda, bfloat16
import os
import transformers
model_id = 'meta-llama/Llama-2-13b-chat-hf'

Quantization techniques reduce memory and computational costs by representing weights and activations with lower-precision data types like 8-bit integers (int8). This enables loading larger models you normally wouldn’t be able to fit into memory, and thus speeds up inference.
To make the process of model quantization more accessible, Hugging Face has seamlessly integrated with the [Bitsandbytes](https://huggingface.co/docs/accelerate/usage_guides/quantization) library.

Define a config from `Bitsandbytes` that enables 4-bit quantization and set the nested quantization to `true`. This changes the datatype from float 32 (default) to normalized float 4 datatype to contain 4 bits of information.
Additionally, add a compute type to store weights in 4-bits, but the computation to happen in 16-bit (bfloat16).
Moreover, set the `bnb_4bit_use_double_quant` to true, which uses a second quantization after the first one to save an additional 0.4 bits per parameter.
Refer to [here](https://huggingface.co/docs/transformers/main_classes/quantization) for more information.

In [8]:
  ### your code ###
bitsAndBites_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=bfloat16,
    bnb_4bit_use_double_quant=True)
  ### your code ###

Use your Hugging Face token to load the correct model configuration using the `transformers` library.

In [9]:
### your code ###
model_config = transformers.AutoConfig.from_pretrained(model_id, token=os.getenv('HF_AUTH'))
### your code ###


Load the model for text generation (pay attention to the model type) using the configuration file you have defined, with the specified quantization, and set the `trust_remote_code` flag to `true`. Another flag that is useful for large mode is  `device_map="auto"`. By setting this flag, Accelerate will determine where to put each layer to maximize the use of GPUs and offload the rest on the CPU, or even the hard drive if you don’t have enough GPU RAM (or CPU RAM).

It will take a while for the model to download.

In [10]:
#Loading the model will take some time, (roughly 5 min)
### your code ###
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    config=model_config,
    quantization_config=bitsAndBites_config,
    trust_remote_code=True,
    device_map="auto",
    token=os.getenv('HF_AUTH'))
### your code ###
model.eval()
print(f"Model loaded ")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Model loaded 


You can even check the memory footprint of your model using the `get_memory_footprint` method.


In [ ]:
model.get_memory_footprint()

7083970560

The next thing we need to do is initialize a `text-generation` pipeline with Hugging Face that uses the Llama2 model to generate some text, given some input. We will then use this pipeline inside LangChain to build our question-answering system.
`text-generation` pipeline generates text from a language model conditioned on a given input. The pipeline is similar to other Hugging Face pipelines and requires two things that we must initialize:

1.   A language model, in this case, it will be `meta-llama/Llama-2-13b-chat-hf`.
2.   A tokenizer for the language model.

LangChain expects the full-text outputs, therefore set the `return_full_text` to true. You can also pass additional generation parameters to the model.
Since we want the questions to be answered mainly based on the retrieved chunks, let's set the model temperature to a low value of 0.01 to reduce randomness. Additionally, add a repetition penalty of 1.1 to stop the model from repeating itself and the maximum number of generation tokens to 512.

In [11]:
### your code ###
generate_text = transformers.pipeline(
    task='text-generation',
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,
    temperature=0.01,
    repetition_penalty=1.1,
    max_new_tokens=512)

### your code ###

We provide the language model a general question to make sure our pipeline is working correctly.

In [ ]:
sample_input="Explain to me the difference between alligator and crocodile."
### your code ###
generated_text=generate_text(sample_input)
### your code ###
print(generated_text)

[{'generated_text': 'Explain to me the difference between alligator and crocodile.\nAlligators and crocodiles are both large, carnivorous reptiles that live in wetlands and rivers, but there are several key differences between them. Here are some of the main differences:\n\n1. Appearance: Alligators have a wider, rounder snout compared to crocodiles, which have a longer, thinner snout. Alligators also have a more rounded body shape and shorter legs than crocodiles.\n2. Habitat: Alligators are found only in freshwater environments such as lakes, rivers, and swamps, while crocodiles can be found in both freshwater and saltwater environments.\n3. Geographic range: Alligators are only found in the southeastern United States and China, while crocodiles are found in many parts of the world, including Africa, Asia, Australia, and the Americas.\n4. Behavior: Alligators are generally less aggressive than crocodiles and tend to avoid confrontations with humans. Crocodiles, on the other hand, are

Use the LangChain Hugging Face wrapper, as subset of [LLM chain](https://python.langchain.com/docs/modules/chains/foundational/llm_chain) to create an interface for the text generation pipeline.

In [12]:
### your code ###
from langchain.llms import HuggingFacePipeline
llm = HuggingFacePipeline(pipeline=generate_text)
### your code ###

To confirm that it works the same way, use the sample input to generate text using the llm chain. The input should be passed as the `prompt` to the language model.

In [ ]:
### your code ###
llm(prompt=sample_input)
### your code ###

/root/miniconda3/envs/nlp/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'\nAlligators and crocodiles are both large, carnivorous reptiles that live in wetlands and rivers, but there are several key differences between them. Here are some of the main differences:\n\n1. Appearance: Alligators have a wider, rounder snout compared to crocodiles, which have a longer, thinner snout. Alligators also have a more rounded body shape and shorter legs than crocodiles.\n2. Habitat: Alligators are found only in freshwater environments such as lakes, rivers, and swamps, while crocodiles can be found in both freshwater and saltwater environments.\n3. Geographic range: Alligators are only found in the southeastern United States and China, while crocodiles are found in many parts of the world, including Africa, Asia, Australia, and the Americas.\n4. Behavior: Alligators are generally less aggressive than crocodiles and tend to avoid confrontations with humans. Crocodiles, on the other hand, are known for their aggressive behavior and have been responsible for many human att

#### ${\color{red}{Comments\ 1.3}}$

${\color{red}{⚠️Comments\ begin⚠️}}$


```
- 1 point the parameters for `bitsAndBites_config` are correct.
- 0.5 point the configuration file is correctly initlized.
- 1 point the generation model is correct.
- 0.5 point question is answered using the Hugging Face pipeline.
- 0.5 point the LLM chain containig the Hugging Face pipeline is correct.
- 0.5 point question is answered using LLM chain.
-----
4 points
```


${\color{red}{⚠️Comments\ end⚠️}}$

## Subtask 1.4: Question Answering Chain


For Retrieval Augmented Generation (RAG) in LangChain, we need to initialize either a `RetrievalQA` or `RetrievalQAWithSourcesChain` object.

`RetrievalQA` is a method for question-answering tasks, utilizing an index to retrieve relevant documents or text chunks, it is suitable for straightforward Q&A applications.

`RetrievalQAWithSourcesChain` is an extension of RetrievalQA that chains together multiple sources of information, providing context and the source for answers.

 For both of these, we need an LLM and a Pinecone index. For LangChain to be able to use the Pinecone index, we need to initialize it through the LangChain vector store.

 **Hint**: You need to explicitly tell the vector storage where to find the original text.

In [ ]:
from langchain.vectorstores import Pinecone
### your code ###
vectorstore = Pinecone(index, embed_model.embed_query, "text")
### your code ###

/root/miniconda3/envs/nlp/lib/python3.10/site-packages/langchain_community/vectorstores/pinecone.py:75: UserWarning: Passing in `embedding` as a Callable is deprecated. Please pass in an Embeddings object instead.
  warnings.warn(


Let's try a query that is specific to the LangChain documentation and see which chunks are relevant. Use the vector storage defined above to find the top-3 chunks related to the given query.

In [ ]:
query = 'what is a LangChain Agent?'
### your code ###
vectorstore.similarity_search(
    query,  # our search query
    k=3  # return 3 most relevant docs
)
### your code ###

[Document(page_content='langchain 0.1.3¶\nlangchain.agents¶\nAgent is a class that uses an LLM to choose a sequence of actions to take.\nIn Chains, a sequence of actions is hardcoded. In Agents,\na language model is used as a reasoning engine to determine which actions\nto take and in which order.\nAgents select and use Tools and Toolkits for actions.\nClass hierarchy:\nBaseSingleActionAgent --> LLMSingleActionAgent\n                          OpenAIFunctionsAgent\n                          XMLAgent\n                          Agent --> <name>Agent  # Examples: ZeroShotAgent, ChatAgent\nBaseMultiActionAgent  --> OpenAIMultiFunctionsAgent\nMain helpers:\nAgentType, AgentExecutor, AgentOutputParser, AgentExecutorIterator,\nAgentAction, AgentFinish\nClasses¶\nagents.agent.Agent\n[Deprecated]  Agent that calls the language model and deciding the action.\nagents.agent.AgentExecutor\nAgent that is using tools.\nagents.agent.AgentOutputParser\nBase class for parsing agent output into agent acti

Now use the `vectorstore` and `llm` to initialize the `RetrievalQA` object, which showcases question answering over an index. `RetrievalQA` is a document chain, these are useful for summarizing documents, answering questions about documents, extracting information from documents, and more. All such chains operate with 4 different chain types:


1.   `stuff`: it takes a list of documents, inserts them all into a prompt, and passes that prompt to an LLM.
2.   `refine`: it constructs a response by looping over the input documents and iteratively updating its answer. It is well-suited for tasks that require analyzing more documents than can fit in the model’s context.
3. `map_reduce`:  it first applies an LLM chain to each document individually (the Map step), treating the chain output as a new document. It then passes all the new documents to a separate combined documents chain to get a single output (the Reduce step).
4. `map_re_rank`: it runs an initial prompt on each document that not only tries to complete a task but also gives a score for how certain it is in its answer. The highest-scoring response is returned.

For this assignment, we focus only on the first type. Make sure to set the `verbose` to `true`, so we can see the different stages of processing that happens while answering a question (you might need to set this parameter more than once). As mentioned before, we want our retrieve to input top-5 most similiar chunks to the query to generate an answer.

In [ ]:
from langchain.chains import RetrievalQA
### your code ###

rag_pipeline = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(k=5),
    verbose=True
)

### your code ###
query='what is a LangChain Agent?'

First, we try to answer the question only using Llama2. As you see the answer is not convincing as it does not have access to the LangChain documentation.

In [ ]:
llm(query)

'\n\nA LangChain Agent is an intelligent agent that uses natural language processing (NLP) and machine learning (ML) techniques to assist users in finding relevant information on the web. It is designed to help users navigate the vast amount of information available online by providing personalized recommendations and answers to their questions.\n\nThe name "LangChain" refers to the idea of linking together different languages and knowledge sources to create a comprehensive and coherent view of the world. The agent is able to understand and respond to user queries in multiple languages, and it can draw upon a wide range of sources, including text, images, videos, and other forms of media, to provide accurate and relevant results.\n\nSome of the key features of a LangChain Agent include:\n\n1. Natural Language Processing (NLP): The agent is able to understand and interpret natural language queries, allowing users to ask questions in everyday language.\n2. Machine Learning (ML): The agen

Now use the Pipeline from above and see how the answer changes.

In [ ]:
### your code ###
rag_pipeline.run(query)
### your code ###

/root/miniconda3/envs/nlp/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new RetrievalQA chain...

> Finished chain.


' A LangChain Agent is a piece of software that uses a Language Model (LLM) to perform tasks. It is a class that uses an LLM to choose a sequence of actions to take. In Chains, a sequence of actions is hardcoded. In Agents, a language model is used as a reasoning engine to determine which actions to take and in which order.'

#### ${\color{red}{Comments\ 1.4}}$

${\color{red}{⚠️Comments\ begin⚠️}}$


```
- 0.5 point correctly initialize a vector store.
- 1 point performing the top-3 similiarity search.
- 1 point correct initialization of the `rag_pipeline` with all the parameters. If the verbose is not set correctly remove only 0.25.
- 0.5 correct use of the pipeline to answer the question.
-----
3 points
```


${\color{red}{⚠️Comments\ end⚠️}}$

## Subtask 1.5: Conversational Retrieval Chain




We can also extend our retrieval chain to be able to remember the previous questions and answer the current question by looking at the previous context.
The important part of a conversational model is conversation memory, which transforms the stateless language model to be able to remember previous interactions, e.g., similiar to ChatGPT. In this subtask, we will use LangChain to create a conversational memory.


To implement the memory we use `ConversationalRetrievalChain`.
This chain takes in chat history (a list of messages) and new questions and then returns an answer to that question. The algorithm for this chain consists of three parts:

1. Use the chat history and the new question to create a new question that contains the information from the previous context.

2. This new question is passed to the retriever and relevant documents are returned.

3. The retrieved documents are passed to an LLM to generate a final response.

In [ ]:
from langchain.chains import ConversationalRetrievalChain
chat_history = []

### your code ###
qa_conversation =  ConversationalRetrievalChain.from_llm(
    llm,
    vectorstore.as_retriever(k=5),
    verbose=True
)
result = qa_conversation({"question": query, "chat_history": chat_history})
### your code ###




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

langchain 0.1.3¶
langchain.agents¶
Agent is a class that uses an LLM to choose a sequence of actions to take.
In Chains, a sequence of actions is hardcoded. In Agents,
a language model is used as a reasoning engine to determine which actions
to take and in which order.
Agents select and use Tools and Toolkits for actions.
Class hierarchy:
BaseSingleActionAgent --> LLMSingleActionAgent
                          OpenAIFunctionsAgent
                          XMLAgent
                          Agent --> <name>Agent  # Examples: ZeroShotAgent, ChatAgent
BaseMultiActionAgent  --> OpenAIMultiFunctionsAgent
Main helpers:
AgentType, AgentExecutor, AgentOutputParser, AgentExecutorIterator,
AgentAction, AgentFinish
Classes¶
agents

In [ ]:
result["answer"]

' A LangChain Agent is a piece of software that uses a Language Model (LLM) to perform tasks. It is a class that inherits from the BaseSingleActionAgent or BaseMultiActionAgent classes, and it has a number of methods for selecting and using Tools and Toolkits to perform actions. The Agent selects and uses Tools and Toolkits for actions.'

Change the chat history to contain the previous question and answer pair and ask a follow-up question.  

In [ ]:
follow_up="What are tools and toolkits?"
### your code ###
chat_history.append((query, result["answer"]))
result = qa_conversation({"question": follow_up, "chat_history": chat_history})
### your code ###



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: what is a LangChain Agent?
Assistant:  A LangChain Agent is a piece of software that uses a Language Model (LLM) to perform tasks. It is a class that inherits from the BaseSingleActionAgent or BaseMultiActionAgent classes, and it has a number of methods for selecting and using Tools and Toolkits to perform actions. The Agent selects and uses Tools and Toolkits for actions.
Follow Up Input: What are tools and toolkits?
Standalone question:

> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

langchain 0.1.3¶
langchain.agents¶
Agen

This is the previous context that was fed in alongside the new question.

In [ ]:
chat_history

[('what is a LangChain Agent?',
  ' A LangChain Agent is a piece of software that uses a Language Model (LLM) to perform tasks. It is a class that inherits from the BaseSingleActionAgent or BaseMultiActionAgent classes, and it has a number of methods for selecting and using Tools and Toolkits to perform actions. The Agent selects and uses Tools and Toolkits for actions.')]

The current question is answered by knowing that the tools and toolkits are referring to a LangChain Agent, which was part of the previous question.

In [ ]:
result['answer']

'\nTools and toolkits are used to give the agent the capability to access, interact with, and manipulate external resources such as local and remote file systems, APIs, and databases.'

#### ${\color{red}{Comments\ 1.5}}$

${\color{red}{⚠️Comments\ begin⚠️}}$


```
- 1 point to correctly initialize the converstional chain.
- 0.5 point for updating the history and asking a follow up question.
---- 
1.5 points
```


${\color{red}{⚠️Comments\ end⚠️}}$

## **Task 2: Advanced RAG Techniques and Evaluation (4 + 5 = 9 points)**

Now that you have successfully implemented your first RAG system, we dive into more advanced techniques and learn how to evaluate your methods using metrics you learned during the lecture. We focus on evaluation with an already annotated dataset. To this end, we load a small subset of [NarrativeQA](https://huggingface.co/datasets/narrativeqa), which is an English-language dataset of stories and corresponding questions designed to test reading comprehension, especially on long documents. We only load 30 samples from the data, as you will see in the upcoming sections, answer generation takes quite some time. In actual setting, it is advised to use a much larger set to obtain statistically significant results.

In [13]:
from datasets import load_dataset
dataset = load_dataset("satyaalmasian/narrativeqa_subset",split="train[:30]")
len(dataset)

30

Since we already used our free index in Pinecone for the previous task, we use Chroma, an open-source vector database, instead. As opposed to Pinecone, Chroma creates a collection on your machine.

In [14]:
from langchain.docstore.document import Document
documents=[ doc["text"] for doc in dataset["document"]]
questions=[quest for quest in dataset["question"]]
answers=[ans for ans in dataset["answers"]]
documents=list(set(documents))

In [15]:
docs= [Document(page_content=doc, metadata={"source": "local"}) for doc in documents]

The number of documents is smaller  than the number of questions and answers and each document is used as a reference for multiple questions:

In [16]:
print(len(docs))
print(len(questions))

2
30


### Subtask 2.1: Build Contextual Compression in LangChain

Let's split our documents using the TextSplitter from Task 1 and embed them inside the Chroma database with the embedding model of the previous task.

In [17]:
### your code ###
all_splits = text_splitter.split_documents(docs)
### your code ###

In [18]:
from langchain.vectorstores import Chroma
### your code ###
vectordb = Chroma.from_documents(all_splits, embed_model)
retriever = vectordb.as_retriever()
### your code ###

In [19]:
print("Fist question in the set:",questions[2]['text'])
r_docs = retriever.get_relevant_documents(questions[2]['text'])
r_docs

Fist question in the set: Why do more students tune into Mark's show?


[Document(page_content="Reporter #2 - Are you on drugs?\n\nPaige - Arrrgh. Talk Hard. Arrrrrgh.\n\nMark - I've got a lot of homework I'm gonna take off alright.\n\nMarla - Mark I know why your really going home. It's because you wanna listen to that \nshow tonight don't you?\n\n<Play Peter Murphy>\n\n<Nora goes to Marks house where she finds him burning his Happy Harry Hardon \nletters>\n\nNora - Hi! What are you doing? You having fun?\n\nMark - Yeah.\n\nNora - Hey, look I took some of these off the wall for you. I mistakingly thought you \nmight want them.\n\nMark - Thanks.\n\nNora - So I guess you're not going on tonight.\n\nMark - Brilliant.\n\nNora - Is this all just a game to you. You know you can't just shout fire in a theatre and \nwalk out. You have a responsibility for the people who believe in you. What is this? \nC'mon say something, say anything. Open your mouth and say get the hell out of here \nbitch.\n\nMark - I can't.\n\nNora - You can't what?\n\nMark - I can't talk.\n\

First, make a simple RAG pipeline that works on top of the Chroma retriever. This retriever should be similar to the previous task. However, since we want to use it for a large number of questions, remove the `verbose` parameters.

In [20]:
from langchain.chains import RetrievalQA
### your code ###
rag_simple = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever
)
### your code ###

We look at an example question and compare the answer by RAG to the gold answer from the dataset. Note that the answers can contain multiple lines.

In [21]:
rag_simple(questions[2]['text']) #ignore the warning

/root/miniconda3/envs/nlp/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


{'query': "Why do more students tune into Mark's show?",
 'result': " Because he's not afraid to speak his mind and talk about real issues that matter to teens."}

In [22]:
answers[2]

[{'text': 'Mark talks about what goes on at school and in the community.',
  'tokens': ['Mark',
   'talks',
   'about',
   'what',
   'goes',
   'on',
   'at',
   'school',
   'and',
   'in',
   'the',
   'community',
   '.']},
 {'text': 'Because he has a thing to say about what is happening at his school and the community.',
  'tokens': ['Because',
   'he',
   'has',
   'a',
   'thing',
   'to',
   'say',
   'about',
   'what',
   'is',
   'happening',
   'at',
   'his',
   'school',
   'and',
   'the',
   'community',
   '.']}]

Apply the `rag_simple` pipeline to all the question in your corpus and accumulate the answers. **It should take around 10 minutes on a T4 GPU on Colab**.

In [23]:
simple_answers=[]
### your code ###
for question in questions:
    simple_answers.append(rag_simple(question['text'])['result'])
### your code ###

/root/miniconda3/envs/nlp/lib/python3.10/site-packages/transformers/pipelines/base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/root/miniconda3/envs/nlp/lib/python3.10/site-packages/transformers/pipelines/base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/root/miniconda3/envs/nlp/lib/python3.10/site-packages/transformers/pipelines/base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/root/miniconda3/envs/nlp/lib/python3.10/site-packages/transformers/pipelines/base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/root/miniconda3/envs/nlp/lib/python3.10/site-packages/transformers/pipeline

Libraries such as LangChain and [Llamaindex](https://www.llamaindex.ai/) provide a variety of retrieval strategies for building a RAG system. In this subtask, you will use one of these variations called **contextual compression**. This method aims to extract only the relevant information from documents, reducing the need for expensive language model calls and improving response quality. Contextual compression consists of two parts:


1.  **Base retriever:** retrieves the initial set of documents based on the query. This is similar to the retriever from the previous task.
2.   **Document compressor:** processes these documents to extract the relevant content. We use `LLMChainExtractor`, which will iterate over the initially returned documents and extract from each only the content that is relevant to the query.


In [25]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor,LLMChainFilter
from langchain.llms import OpenAI

### your code ###

compression_retriever = ContextualCompressionRetriever(
    base_retriever=retriever,
    base_compressor=LLMChainExtractor.from_llm(llm),
)
### your code ###

Let's take a look at an example of compression retriever works.

In [27]:
print("Fist question in the set:",questions[2]['text'])
compressed_docs = compression_retriever.get_relevant_documents(questions[2]['text'])
compressed_docs

Fist question in the set: Why do more students tune into Mark's show?


/root/miniconda3/envs/nlp/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/root/miniconda3/envs/nlp/lib/python3.10/site-packages/transformers/pipelines/base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/root/miniconda3/envs/nlp/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/root/miniconda3/envs/nlp/lib/python3.10/site-packages/transformers/pipelines/base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/root/miniconda3/envs/nlp/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse

[Document(page_content='* "Why do more students tune into Mark\'s show?"\n* "Mark\'s show"\n* "students"', metadata={'source': 'local'}),
 Document(page_content='* Nora got expelled\n* Nora has been cutting lessons\n* Creswood is mentioned as a staff member', metadata={'source': 'local'}),
 Document(page_content='* "the kids who need the most help like those with drug problems"\n* "this radio person"', metadata={'source': 'local'}),
 Document(page_content='* "more students"\n* "tune into Mark\'s show"\n* "decent grades"\n* "leave me alone"', metadata={'source': 'local'})]

Look at the output and try out several different questions by yourself. Does the compressed output make sense?  
- For QUESTION compression, I think it works. For example, as currently printed out, Mark\'s show and students are duplicated. This should be good for retriever.
- For document compression, the quality of compression varies. Some sentences effectively capture the main idea, while others lose key context.

Compare this to the previous **simple** approach. Which one, in your opinion, is better?
- Depends on the quality of the compression

Finally, we use the new retriever with the Llama2 model from the previous task to create the context compressor RAG pipeline. The code below should be similiar to what you did in the previous task. Once again, make sure to turn off the `verbose` argument.

In [28]:
### your code ###
from langchain.chains import RetrievalQA

rag_compressor = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=compression_retriever
)
### your code ###


In [29]:
rag_compressor(questions[2]['text'])

/root/miniconda3/envs/nlp/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/root/miniconda3/envs/nlp/lib/python3.10/site-packages/transformers/pipelines/base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/root/miniconda3/envs/nlp/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/root/miniconda3/envs/nlp/lib/python3.10/site-packages/transformers/pipelines/base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/root/miniconda3/envs/nlp/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse

{'query': "Why do more students tune into Mark's show?",
 'result': ' Based on the information provided, it seems that Mark\'s show may be popular among students who are struggling with issues such as drug problems or poor academic performance. The mention of Nora getting expelled and cutting lessons suggests that these students may be seeking help or support from Mark\'s show. Additionally, the phrase "the kids who need the most help" and the reference to "those with drug problems" suggest that Mark\'s show may be providing a valuable resource for these students.'}

Now we can use the pipeline to generate answers for all the questions in our dataset. **It should take around 20 minutes on a T4 GPU on Colab.**

In [30]:
compressor_answers=[]
### your code ###
for question in questions:
    compressor_answers.append(rag_compressor(question['text'])['result'])
### your code ###


/root/miniconda3/envs/nlp/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/root/miniconda3/envs/nlp/lib/python3.10/site-packages/transformers/pipelines/base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/root/miniconda3/envs/nlp/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/root/miniconda3/envs/nlp/lib/python3.10/site-packages/transformers/pipelines/base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/root/miniconda3/envs/nlp/lib/python3.10/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse

#### ${\color{red}{Comments\ 2.1}}$

${\color{red}{⚠️Comments\ begin⚠️}}$


```
- 0.5 point the text is correctly split.
- 1 point initializing Chroma db as a retreiever and feeding the documents.
- 0.5 point simple RAG pipline.
- 0.25 point generating answers with simple RAG.
- 1 point the correct compressor retriever.
- 0.5 point compressor RAG pipline.
- 0.25 point generating answers with compressor RAG.
-----
4 points
```


${\color{red}{⚠️Comments\ end⚠️}}$

### Subtask 2.2. Evaluate

Since we have access to ground truth answers, we can use various evaluation metrics from the literature. In this task, we explore three metrics:


1.   **BLEU:** BLEU score stands for Bilingual Evaluation Understudy and is a precision-based metric developed
for evaluating machine translation. BLEU scores a candidate by computing the
number of n-grams in the candidate that also appear
in a reference. The n can vary, in this task we compute for n=4.
2.   **ROUGE:** ROUGE score stands for Recall-Oriented Understudy for Gisting Evaluation and is an F-measure metric designed for
evaluating translation and summarization. There are a number of variants of ROUGE.
3. **BERTScore:** BERTScore first obtains BERT representation of each word in the candidate and reference by feeding the candidate
and reference through a BERT model separately.
An alignment is then computed between candidate
and reference words by computing pairwise cosine
similarity. This alignment is then aggregated in to
precision and recall scores before being aggregated
into a (modified) F1 score that is weighted using
inverse-document-frequency values.

Luckily, Hugging Face has an implementation for all these metrics. Use the `evaluate` library to load the metrics.

Use the loaded metrics to compare the RAG pipelines from the previous subtask.

In [31]:
import evaluate
### your code ###
bleu = evaluate.load('bleu')
rouge = evaluate.load('rouge')
bertscore = evaluate.load('bertscore')
### your code ###

As seen in the previous subtask, the answers can contain multiple lines. To be able to compare the output of our systems to the gold answers, merge the multiple answers into a single string.

In [32]:
answers_merged=[]
### your code ###
for answer_list in answers:
    # Join all texts in the answer list into a single string
    merged_answer = ' '.join([ans['text'] for ans in answer_list])
    answers_merged.append(merged_answer)

### your code ###
print(len(answers_merged))

30


Compute the BLUE score for the simple RAG and compressor RAG.

In [35]:
### your code ###
bleu_simple = bleu.compute(predictions=simple_answers, references=answers_merged)
bleu_compressor = bleu.compute(predictions=compressor_answers, references=answers_merged)
### your code ###
print("Simple system:")
print(bleu_simple)
print("Compressor:")
print(bleu_compressor)

Simple system:
{'bleu': 0.0, 'precisions': [0.11329305135951662, 0.004746835443037975, 0.0016611295681063123, 0.0], 'brevity_penalty': 1.0, 'length_ratio': 2.2364864864864864, 'translation_length': 662, 'reference_length': 296}
Compressor:
{'bleu': 0.0, 'precisions': [0.09887005649717515, 0.008849557522123894, 0.0030864197530864196, 0.0], 'brevity_penalty': 1.0, 'length_ratio': 2.391891891891892, 'translation_length': 708, 'reference_length': 296}


What does the elements below in the output of the BLEU impelementation in Hugging Face mean? (do not copy and paste the documentation but write the implications behind each element!).



1.   **precisions:** `your answer`
2.   **brevity_penalty:** `your answer`
3.   **translation_length:** `your answer`
4.   **reference_length:** `your answer`
5.   **length_ratio:** `your answer`




**Answer:**


1.   **precisions:** This represents the precision scores for different n-gram lengths, usually ranging from 1 to 4. Precision in this context measures how many of the n-grams in the machine-translated text are also found in the reference translations. Higher precision scores indicate a greater overlap with the reference texts, suggesting better translation quality. However, precision alone can favor shorter translations, which is why BLEU also incorporates a brevity penalty.
2.   **brevity_penalty:** The brevity penalty addresses the limitation of precision by penalizing overly short translations. If the machine-generated translation is shorter than the reference texts, the brevity penalty will be less than 1, reducing the overall BLEU score. This encourages translations to not only be accurate at the n-gram level but also to match the length of the reference texts more closely.
3.   **translation_length:**   This is the total length of the machine-translated text. It's used to calculate the brevity penalty and to understand the scale of the translation task. In general, longer texts pose more challenges for maintaining consistency and accuracy throughout.
4.   **reference_length:**  This refers to the length of the reference translations that the machine-generated text is being compared against. It's crucial for calculating the brevity penalty and for establishing a baseline for the length that a high-quality translation should achieve.
5. **length_ratio:** This is the ratio of the translation length to the reference length. It provides a quick insight into how the length of the translated text compares to the reference. A ratio close to 1 indicates that the translation length closely matches the reference length, which is ideal in many contexts. Ratios significantly above or below 1 might suggest that the translation is respectively verbose or too concise, potentially affecting readability and fidelity to the original meaning.

In [36]:
### your code ###
rouge_simple = rouge.compute(predictions=simple_answers, references=answers_merged)
rouge_compressor = rouge.compute(predictions=compressor_answers, references=answers_merged)
### your code ###
print("Simple system:")
print(rouge_simple)
print("Compressor:")
print(rouge_compressor)

Simple system:
{'rouge1': 0.11591846505131298, 'rouge2': 0.009312169312169312, 'rougeL': 0.10273551724919033, 'rougeLsum': 0.10339797821345781}
Compressor:
{'rouge1': 0.10965082465623792, 'rouge2': 0.014217006806238507, 'rougeL': 0.09819283588949361, 'rougeLsum': 0.09786033247272342}


What is the difference in variants of ROUGE (ROUGE-N, ROUGE-L, ROUGE-SUM)?

`your answer`


**Answer:**

ROUGE measures the similarity between the machine-generated summary and the reference summaries using overlapping n-grams, word sequences that appear in both the machine-generated summary and the reference summaries. The most common n-grams used are unigrams, bigrams, and trigrams. ROUGE score calculates the recall of n-grams in the machine-generated summary by comparing them to the reference summaries.

**ROUGE-N:** This variant focuses on the overlap of n-grams (contiguous sequences of 'n' words) between the generated text and the reference texts. The most common is ROUGE-1 (for unigrams) and ROUGE-2 (for bigrams), which measure the overlap of single words and two-word phrases, respectively. ROUGE-N is useful for assessing the surface similarity of the generated text to the references, emphasizing the importance of choosing the right words and phrases.

**ROUGE-L:**This variant measures the Longest Common Subsequence (LCS) between the generated text and the reference texts. Unlike ROUGE-N, which requires exact matches of n-gram sequences, ROUGE-L considers the longest sequence of words that appears in both the generated and reference texts in the same order, allowing for non-contiguous matches. This can better capture the fluency and the overall structure of the content, as it reflects the ability to preserve significant phrases even if the exact wording differs.

**ROUGE-S:** This is an extension of ROUGE that is used specifically for evaluating the summary content by considering multiple reference summaries and aggregating their individual ROUGE scores. It's particularly useful in scenarios where there are multiple valid ways to summarize a given text, and it allows for a more comprehensive evaluation by taking into account the variability among reference summaries.


In [37]:
import numpy as np
bertscore_simple_averaged={}
bertscore_compressor_averaged={}
### your code ###
results_simple = bertscore.compute(predictions=simple_answers, references=answers_merged, lang="en")
bertscore_simple_averaged['precision'] = np.mean(results_simple['precision'])
bertscore_simple_averaged['recall'] = np.mean(results_simple['recall'])
bertscore_simple_averaged['f1'] = np.mean(results_simple['f1'])
results_compressor = bertscore.compute(predictions=compressor_answers, references=answers_merged, lang="en")
bertscore_compressor_averaged['precision'] = np.mean(results_compressor['precision'])
bertscore_compressor_averaged['recall'] = np.mean(results_compressor['recall'])
bertscore_compressor_averaged['f1'] = np.mean(results_compressor['f1'])
### your code ###
print("Simple system:")
print(bertscore_simple_averaged)
print("Compressor:")
print(bertscore_compressor_averaged)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Simple system:
{'precision': 0.8405957301457723, 'recall': 0.8556291083494822, 'f1': 0.8479224026203156}
Compressor:
{'precision': 0.8360168476899464, 'recall': 0.851251111427943, 'f1': 0.8434217433134715}


Which model works better?
- In terms of results, the Simple system is a little better.

#### ${\color{red}{Comments\ 2.2}}$

${\color{red}{⚠️Comments\ begin⚠️}}$


```
- 0.5 point loading the metrics.
- 0.5 point parsing the answers.
- 0.5 point computation of BLEU.
- 0.25 *5 = 1.25 points  meaning of each part of BLEU score.
- 0.5 point computation of ROUGE.
- 0.25 *3= 0.75 point  variants for ROUGE.
- 1 point computation of BERTScore.
-----
3.5 points
```


${\color{red}{⚠️Comments\ end⚠️}}$